In [2]:
import pandas as pd
import json

In [3]:
with open("ccpp_parsed.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index")

In [4]:
with open("ccpp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index")

In [5]:
key = "connections"

dfs_tespy[key]["fluid_group"] = None
dfs_ebs[key]["fluid_group"] = None

groups_compositions = {}

for idx, row in dfs_tespy[key].iterrows():
    rounded_fluids = {
        k: round(v, 4) for k, v in row["mass_composition"].items()
    }
    dfs_tespy[key].loc[idx, "fluid_group"] = f"{rounded_fluids}"

dfs_tespy[key]["dh"] = 0.0
dfs_tespy[key]["ds"] = 0.0
for group in dfs_tespy[key]["fluid_group"].unique():
    mask = dfs_tespy[key]["fluid_group"] == group
    # reference_df = dfs_tespy[key].loc[mask, ["h", "s"]].copy()
    # locator = reference_df.index.astype(int).min()
    dfs_tespy[key].loc[mask, ["dh", "ds"]] = (dfs_tespy[key].loc[mask, ["h", "s"]].iloc[0] - dfs_tespy[key].loc[mask, ["h", "s"]]).values


groups_compositions = {}

for idx, row in dfs_ebs[key].iterrows():
    if type(row["mass_composition"]) == dict:
        rounded_fluids = {
            k: round(v, 4) for k, v in row["mass_composition"].items()
        }
        dfs_ebs[key].loc[idx, "fluid_group"] = f"{rounded_fluids}"
    else:
        continue

dfs_ebs[key]["dh"] = 0.0
dfs_ebs[key]["ds"] = 0.0
for group in dfs_ebs[key]["fluid_group"].unique():
    if group is not None:
        mask = dfs_ebs[key]["fluid_group"] == group
        reference_df = dfs_ebs[key].loc[mask, ["h", "s"]].copy()
        locator = reference_df.index.astype(int).min().astype(str)
        dfs_ebs[key].loc[mask, ["dh", "ds"]] = (dfs_ebs[key].loc[mask, ["h", "s"]].loc[locator] - dfs_ebs[key].loc[mask, ["h", "s"]]).values

In [6]:
columns = ["m", "p", "T"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    dfs_ebs[key].loc[overlapping_index, columns]
    - dfs_tespy[key].loc[overlapping_index, columns]
).abs() / dfs_tespy[key].loc[overlapping_index, columns]
diff_to_ebs = (
    dfs_ebs[key].loc[overlapping_index, columns]
    - dfs_tespy[key].loc[overlapping_index, columns]
).abs() / dfs_tespy[key].loc[overlapping_index, columns]

columns = ["dh", "ds"]
diff_to_ebs[columns] = (
    dfs_ebs[key].loc[overlapping_index, columns]
    - dfs_tespy[key].loc[overlapping_index, columns]
)

In [ ]:
diff_to_ebs.index = diff_to_ebs.index.astype(int)
diff_to_ebs.sort_index()

In [ ]:
assert (diff_to_ebs < 1e-3).all().all()
assert (diff_to_tespy < 1e-3).all().all()